In [1]:
import numpy as np
import pandas as pd
import os
import ray
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset, load_metric
import evaluate
import ray.data
from ray.data.preprocessors import BatchMapper
from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.air.integrations.wandb import WandbLoggerCallback
import wandb

In [2]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/rj2665/.netrc


True

In [3]:
os.environ['RAY_memory_usage_threshold'] = "0.98"

In [4]:
model_name = "distilroberta-base"
use_gpu = True
num_workers = 2
cpus_per_worker = 1

In [5]:
ray.init(
    runtime_env={
        "pip": [
            "datasets",
            "evaluate",
            "accelerate>=0.16.0",
            "transformers>=4.26.0",
            "torch>=1.12.0",
            "deepspeed",
        ]
    }
)

2023-05-09 01:44:40,148	INFO worker.py:1550 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.12
Ray version:,2.3.1
Dashboard:,http://127.0.0.1:8265


In [6]:
imdb_data = load_dataset("imdb")

Found cached dataset imdb (/home/rj2665/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
ray_imdb_data = ray.data.from_huggingface(imdb_data)

In [8]:
ray_imdb_data

{'train': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'test': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'unsupervised': Dataset(num_blocks=1, num_rows=50000, schema={text: string, label: int64})}

In [9]:
def tokenize(batch: pd.DataFrame):
    # Tokenize the input text
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    tokenized_texts = tokenizer(list(batch['text']), padding=True, truncation=True, return_tensors="np")
    tokenized_texts['input_ids'] = tokenized_texts['input_ids'].tolist()
    tokenized_texts['attention_mask'] = tokenized_texts['attention_mask'].tolist()
    tokenized_texts = {**batch, **tokenized_texts}
    return pd.DataFrame.from_dict(tokenized_texts)

In [10]:
tokenizer = BatchMapper(tokenize, batch_format="pandas")

In [11]:
batch_size = 16
def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    torch.backends.cuda.matmul.allow_tf32 = True
    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 8,
        },
        "bf16": {"enabled": "auto"},
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
            },
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            "offload_param": {
                "device": "cpu",
                "pin_memory": True,
            },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 10,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False,
    }
    args = TrainingArguments(
        "ray-bert-finetune-imdb",
        save_strategy="no",
        logging_steps=1,
        learning_rate=config.get("learning_rate", 2e-5),
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        label_names=["input_ids", "attention_mask"],
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        fp16=True,
        # gradient_checkpointing=True,
        deepspeed=deepspeed,
        push_to_hub=False,
        disable_tqdm=True,  # declutter the output a little
        # no_cuda=not use_gpu,  # you need to explicitly set no_cuda if you want CPUs
    )

    def compute_metrics(eval_pred):
        load_accuracy = load_metric("accuracy")
        load_f1 = load_metric("f1")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
        f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
        return {"accuracy": accuracy, "f1": f1}


    trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator = default_data_collator,
    )

    print("Starting training")
    return trainer

In [12]:
trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu, resources_per_worker={"GPU": 1, "CPU": 1}),
    datasets={
        "train": ray_imdb_data["train"],
        "evaluation": ray_imdb_data["test"],
    },
    run_config=RunConfig(
#         callbacks=[MLflowLoggerCallback(experiment_name="ray-roberta-finetune-imdb")],
        callbacks=[WandbLoggerCallback(project="ray-roberta-finetune-imdb")],
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="loss",
            checkpoint_score_order="min",
        ),
    ),
    preprocessor=tokenizer,
)

In [13]:
result = trainer.fit()


2023-05-09 01:45:15,092	INFO wandb.py:314 -- Already logged into W&B.
(_WandbLoggingActor pid=12725) wandb: Currently logged in as: rj2665 (hpml3). Use `wandb login --relogin` to force relogin
(HuggingFaceTrainer pid=12722) 2023-05-09 01:45:24,146	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper] -> AllToAllOperator[randomize_block_order]
(_WandbLoggingActor pid=12725) wandb: Tracking run with wandb version 0.15.2
(_WandbLoggingActor pid=12725) wandb: Run data is saved locally in /home/rj2665/ray_results/HuggingFaceTrainer_2023-05-09_01-45-14/HuggingFaceTrainer_24ef3_00000_0_2023-05-09_01-45-15/wandb/run-20230509_014518-24ef3_00000
(_WandbLoggingActor pid=12725) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=12725) wandb: Syncing run HuggingFaceTrainer_24ef3_00000
(_WandbLoggingActor pid=12725) wandb: ⭐️ View project at https://wandb.ai/hpml3/ray-roberta-finetune-imdb
(_WandbLoggingActor pid=12725) wandb: 🚀 Vie

(RayTrainWorker pid=13733) Is CUDA available: True
(RayTrainWorker pid=13732) Is CUDA available: True


(RayTrainWorker pid=13732) Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
(RayTrainWorker pid=13732) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(RayTrainWorker pid=13732) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(RayTrainWorker pid=13732) Some weights of RobertaForSequenceClassification were not initi

(RayTrainWorker pid=13733) Starting training
(RayTrainWorker pid=13732) Starting training
(RayTrainWorker pid=13733) Installed CUDA version 11.3 does not match the version torch was compiled with 11.7 but since the APIs are compatible, accepting this combination
(RayTrainWorker pid=13732) Installed CUDA version 11.3 does not match the version torch was compiled with 11.7 but since the APIs are compatible, accepting this combination


(RayTrainWorker pid=13732) Using /home/rj2665/.cache/torch_extensions/py37_cu117 as PyTorch extensions root...
(RayTrainWorker pid=13733) Using /home/rj2665/.cache/torch_extensions/py37_cu117 as PyTorch extensions root...
(RayTrainWorker pid=13732) Detected CUDA files, patching ldflags
(RayTrainWorker pid=13732) Emitting ninja build file /home/rj2665/.cache/torch_extensions/py37_cu117/cpu_adam/build.ninja...
(RayTrainWorker pid=13732) Building extension module cpu_adam...
(RayTrainWorker pid=13732) Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


...

(RayTrainWorker pid=13732) {'loss': 0.0343, 'learning_rate': 2.5575447570332484e-07, 'epoch': 1.97}
(RayTrainWorker pid=13732) {'loss': 0.2638, 'learning_rate': 2.4296675191815855e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.3431, 'learning_rate': 2.3017902813299236e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.3101, 'learning_rate': 2.173913043478261e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.3031, 'learning_rate': 2.0460358056265988e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.0563, 'learning_rate': 1.9181585677749362e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.3121, 'learning_rate': 1.7902813299232738e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.07, 'learning_rate': 1.6624040920716116e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.0607, 'learning_rate': 1.534526854219949e-07, 'epoch': 1.98}
(RayTrainWorker pid=13732) {'loss': 0.0777, 'learning_rate': 1.4066496163682866e-07, 'epoch': 1.99}
(Ray

(RayTrainWorker pid=13732) /opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1433: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
(RayTrainWorker pid=13732)   "Positional args are being deprecated, use kwargs instead. Refer to "
(RayTrainWorker pid=13733) /opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1433: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
(RayTrainWorker pid=13733)   "Positional args are being deprecated, use kwargs instead. Refer to "


(RayTrainWorker pid=13732) {'train_runtime': 835.0049, 'train_samples_per_second': 29.94, 'train_steps_per_second': 1.873, 'train_loss': 0.20461657528987015, 'epoch': 2.0}


2023-05-09 02:01:39,259	ERROR trial_runner.py:1062 -- Trial HuggingFaceTrainer_24ef3_00000: Error processing event.
ray.exceptions.RayTaskError(PermissionError): ray::_Inner.train() (pid=12722, ip=10.158.0.3, repr=HuggingFaceTrainer)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py", line 340, in entrypoint
    self._status_reporter.get_checkpoint(),
  File "/opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py", line 505, in _trainable_func
    super()._trainable_func(self._merged_config, reporter, checkpoint_dir)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py", line 654, in _trainable_func
    output = fn()
  File "/opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py", line 415, in train_func
    trainer.training_loop()
  File "/opt/cond

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 result = trainer.fit()                                                                       │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py:368 in fit                      │
│                                                                                                  │
│   365 │   │   try:                                                                               │
│   366 │   │   │   result = result_grid[0]                                                        │
│   367 │   │   │   if result.error:                                                               │
│ ❱ 368 │   │   │   │   raise result.error                                                         │
│   369 │   │   except TuneError as e:                                                             │
│   370 │   │   │   raise TrainingFailedError from e                                               │
│   371 │   │   return result                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RayTaskError(PermissionError): [36mray::_Inner.train()[39m (pid=12722, ip=10.158.0.3, repr=HuggingFaceTrainer)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py", line 340, in entrypoint
    self._status_reporter.get_checkpoint(),
  File "/opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py", line 505, in _trainable_func
    super()._trainable_func(self._merged_config, reporter, checkpoint_dir)
  File "/opt/conda/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py", line 654, in 
_trainable_func
    output = fn()
  File "/opt/conda/lib/python3.7/site-packages/ray/train/base_trainer.py", line 415, in train_func
    trainer.training_loop()
  File "/opt/conda/lib/python3.7/site-packages/ray/train/data_parallel_trainer.py", line 395, in training_loop
    self._report(training_iterator)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/data_parallel_trainer.py", line 342, in _report
    for results in training_iterator:
  File "/opt/conda/lib/python3.7/site-packages/ray/train/trainer.py", line 134, in __next__
    next_results = self._run_with_error_handling(self._fetch_next_result)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/trainer.py", line 97, in _run_with_error_handling
    return func()
  File "/opt/conda/lib/python3.7/site-packages/ray/train/trainer.py", line 178, in _fetch_next_result
    results, decode_checkpoint_fn=self._backend._decode_data
  File "/opt/conda/lib/python3.7/site-packages/ray/train/_internal/checkpoint.py", line 138, in _process_checkpoint
    self.register_checkpoint(checkpoint=tracked_checkpoint)
  File "/opt/conda/lib/python3.7/site-packages/ray/air/_internal/checkpoint_manager.py", line 322, in 
register_checkpoint
    self._process_persistent_checkpoint(persisted_checkpoint)
  File "/opt/conda/lib/python3.7/site-packages/ray/train/data_parallel_trainer.py", line 51, in 
_process_persistent_checkpoint
    checkpoint=checkpoint
  File "/opt/conda/lib/python3.7/site-packages/ray/train/_internal/checkpoint.py", line 235, in 
_process_persistent_checkpoint
    checkpoint.commit(path)
  File "/opt/conda/lib/python3.7/site-packages/ray/air/_internal/checkpoint_manager.py", line 118, in comm